In [1]:
# NOTE
# This notebook demonstrates the pipeline used to download satellite images using the Sentinel Hub API.
# At the time of submission, the Sentinel Hub account credits have expired, so re-running this notebook may result in authentication or quota errors.
# The images used for training and analysis were downloaded earlier and are already stored locally. This notebook is kept for reproducibility and demonstration of the data acquisition process.

In [2]:
import os
print(os.getcwd())

C:\Users\Sameer Kumar\Documents\GitHub\Multimodal-Model-for-House-Price-Prediction\notebooks


In [1]:
import os
import sys
from pathlib import Path

import pandas as pd
import numpy as np

from PIL import Image
import cv2

import matplotlib.pyplot as plt

from tqdm import tqdm


from sentinelhub import (
    SHConfig, MimeType, CRS, BBox,
    SentinelHubRequest, DataCollection
)

import torch
import torchvision
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader

In [2]:
import os
import pandas as pd
from sentinelhub import (
    SHConfig, SentinelHubRequest, DataCollection, MimeType,
    CRS, BBox
)

BASE_PATH = os.path.abspath("..")   # project root
IMAGE_PATH = os.path.join(BASE_PATH, "images")

os.makedirs(IMAGE_PATH, exist_ok=True)

# Load training data
df = pd.read_csv(os.path.join(BASE_PATH, "data", "train.csv"))
df["id"] = df["id"].astype(str)

print("Project root:", BASE_PATH)
print("Images saved to:", IMAGE_PATH)
print("Rows loaded:", len(df))

Project root: C:\Users\Sameer Kumar\Documents\GitHub\Multimodal-Model-for-House-Price-Prediction
Images saved to: C:\Users\Sameer Kumar\Documents\GitHub\Multimodal-Model-for-House-Price-Prediction\images
Rows loaded: 16209


In [3]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,9117000170,20150505T000000,268643,4,2.25,1810,9240,2.0,0,0,...,7,1810,0,1961,0,98055,47.4362,-122.187,1660,9240
1,6700390210,20140708T000000,245000,3,2.50,1600,2788,2.0,0,0,...,7,1600,0,1992,0,98031,47.4034,-122.187,1720,3605
2,7212660540,20150115T000000,200000,4,2.50,1720,8638,2.0,0,0,...,8,1720,0,1994,0,98003,47.2704,-122.313,1870,7455
3,8562780200,20150427T000000,352499,2,2.25,1240,705,2.0,0,0,...,7,1150,90,2009,0,98027,47.5321,-122.073,1240,750
4,7760400350,20141205T000000,232000,3,2.00,1280,13356,1.0,0,0,...,7,1280,0,1994,0,98042,47.3715,-122.074,1590,8071


In [4]:
from sentinelhub import SHConfig

CLIENT_ID = os.getenv("SENTINEL_CLIENT_ID")
CLIENT_SECRET = os.getenv("SENTINEL_CLIENT_SECRET")
INSTANCE_ID = os.getenv("SENTINEL_INSTANCE_ID")

if not all([CLIENT_ID, CLIENT_SECRET, INSTANCE_ID]):
    raise RuntimeError("Sentinel Hub credentials not found in environment variables")


# Sentinel Hub Config
config = SHConfig()
config.sh_client_id     = CLIENT_ID
config.sh_client_secret = CLIENT_SECRET
config.instance_id      = INSTANCE_ID

In [5]:
from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, DataCollection
import pandas as pd
import numpy as np
import os

In [7]:
def fetch_image(lat, lon, pid, size=256):

    bbox = BBox(bbox=[lon-0.001, lat-0.001, lon+0.001, lat+0.001], crs=CRS.WGS84)
    
    request = SentinelHubRequest(
        data_folder=IMAGE_PATH,
        evalscript="""
        // RGB mapping
        return [B04, B03, B02];
        """,
        input_data=[SentinelHubRequest.input_data(data_collection=DataCollection.SENTINEL2_L2A)],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=bbox,
        size=(size, size),
        config=config
    )

    result = request.get_data(save_data=True)

In [8]:
for i, row in df.head(5).iterrows():
    print("Downloading:", row['id'])

    try:
        fetch_image(row['lat'], row['long'], row['id'])
    except Exception as e:
        print("FAILED:", row['id'], e)

Downloading: 9117000170
Downloading: 6700390210
Downloading: 7212660540
Downloading: 8562780200
Downloading: 7760400350


In [9]:
LIMIT = None

In [10]:
df = df[['id', 'lat', 'long']]
df.head()

,id,lat,long
0,9117000170,47.4362,-122.187
1,6700390210,47.4034,-122.187
2,7212660540,47.2704,-122.313
3,8562780200,47.5321,-122.073
4,7760400350,47.3715,-122.074


In [29]:
# Pick first property
lat, lon = df.loc[0, "lat"], df.loc[0, "long"]
prop_id  = str(df.loc[0, "id"])

bbox = BBox(
    bbox=[lon-0.005, lat-0.005, lon+0.005, lat+0.005], 
    crs=CRS.WGS84
)

request = SentinelHubRequest(
    data_folder=os.path.join(IMAGE_PATH, prop_id),
    evalscript="""
    // TRUE COLOR VISUALIZATION
    return [B04, B03, B02];
    """,
    input_data=[SentinelHubRequest.input_data(
        data_collection=DataCollection.SENTINEL2_L2A
    )],
    responses=[SentinelHubRequest.output_response("default", MimeType.JPG)],
    bbox=bbox,
    size=(256, 256),
    config=config
)

data = request.get_data(save_data=True)
print("Test image saved at:", os.path.join(IMAGE_PATH, prop_id))

Test image saved at: C:\Users\Sameer Kumar\Documents\GitHub\Multimodal-Model-for-House-Price-Prediction\notebooks\images\9117000170


In [30]:
from tqdm import tqdm

for idx, row in tqdm(df.iterrows(), total=len(df)):
    lat, lon = row["lat"], row["long"]
    prop_id  = str(row["id"])

    save_dir = os.path.join(IMAGE_PATH, prop_id)
    save_img = os.path.join(save_dir, "default.jpg")

    if os.path.exists(save_img):  # skip if already exists
        continue

    bbox = BBox(
        bbox=[lon-0.005, lat-0.005, lon+0.005, lat+0.005],
        crs=CRS.WGS84
    )

    request = SentinelHubRequest(
        data_folder=save_dir,
        evalscript="return [B04, B03, B02];",
        input_data=[SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A
        )],
        responses=[SentinelHubRequest.output_response("default", MimeType.JPG)],
        bbox=bbox,
        size=(256, 256),
        config=config
    )

    try:
        request.get_data(save_data=True)
    except Exception as e:
        print("Failed for ID:", prop_id, "| Error:", e)

print("DONE: All possible images downloaded")

 90%|██████████████████████████████████████████████████████████████████▉       | 14665/16209 [3:18:14<19:35,  1.31it/s]

Failed for ID: 9828702902 | Error: Failed to download from:
https://services.sentinel-hub.com/api/v1/process
with HTTPError:
403 Client Error: Forbidden for url: https://services.sentinel-hub.com/api/v1/process
Server response: "{"status": 403, "reason": "Forbidden", "message": "Insufficient processing units or requests available in your account. Upgrade account or acquire additional credits.", "code": "ACCESS_INSUFFICIENT_REQUESTS"}"


 90%|██████████████████████████████████████████████████████████████████▉       | 14666/16209 [3:18:15<18:46,  1.37it/s]

Failed for ID: 3630080430 | Error: Failed to download from:
https://services.sentinel-hub.com/api/v1/process
with HTTPError:
403 Client Error: Forbidden for url: https://services.sentinel-hub.com/api/v1/process
Server response: "{"status": 403, "reason": "Forbidden", "message": "Insufficient processing units or requests available in your account. Upgrade account or acquire additional credits.", "code": "ACCESS_INSUFFICIENT_REQUESTS"}"


 90%|██████████████████████████████████████████████████████████████████▉       | 14667/16209 [3:18:16<18:17,  1.41it/s]

...

Failed for ID: 7202350480 | Error: Failed to download from:
https://services.sentinel-hub.com/api/v1/process
with HTTPError:
403 Client Error: Forbidden for url: https://services.sentinel-hub.com/api/v1/process
Server response: "{"status": 403, "reason": "Forbidden", "message": "Insufficient processing units or requests available in your account. Upgrade account or acquire additional credits.", "code": "ACCESS_INSUFFICIENT_REQUESTS"}"


100%|█████████████████████████████████████████████████████████████████████████▉| 16208/16209 [3:35:22<00:00,  1.50it/s]

Failed for ID: 1723049033 | Error: Failed to download from:
https://services.sentinel-hub.com/api/v1/process
with HTTPError:
403 Client Error: Forbidden for url: https://services.sentinel-hub.com/api/v1/process
Server response: "{"status": 403, "reason": "Forbidden", "message": "Insufficient processing units or requests available in your account. Upgrade account or acquire additional credits.", "code": "ACCESS_INSUFFICIENT_REQUESTS"}"


100%|██████████████████████████████████████████████████████████████████████████| 16209/16209 [3:35:23<00:00,  1.25it/s]

Failed for ID: 6147650280 | Error: Failed to download from:
https://services.sentinel-hub.com/api/v1/process
with HTTPError:
403 Client Error: Forbidden for url: https://services.sentinel-hub.com/api/v1/process
Server response: "{"status": 403, "reason": "Forbidden", "message": "Insufficient processing units or requests available in your account. Upgrade account or acquire additional credits.", "code": "ACCESS_INSUFFICIENT_REQUESTS"}"
DONE: All possible images downloaded


In [31]:
downloaded = len(os.listdir(IMAGE_PATH))
print("📌 Total images downloaded:", downloaded)

📌 Total images downloaded: 14599


In [32]:
import os
import pandas as pd

df = pd.read_csv("../data/train.csv")

available_ids = set(os.listdir(IMAGE_PATH))
df['has_image'] = df['id'].astype(str).isin(available_ids)

df_final = df[df['has_image'] == True].reset_index(drop=True)

print("🔹 Original dataset:", len(df))
print("🔹 Usable for multimodal model:", len(df_final))

🔹 Original dataset: 16209
🔹 Usable for multimodal model: 14697
